## Transfer Learning

Transfer learning is the when you use a pre trained model and apply it to the data set that you have. 

I have chosen to use the Resnet 50 model. It is a model developed by the Microsoft Research team. It consists of 50 layers which include 48 convolutional layers and 2 fully connected layers. It contains many different filters that capture the image at different scales. The use of skip connections allows the model to learn residuals between the input layer and output layer. It is a contrast to normal Sequential method. 

The model has been pre trained on Image net as well which is a very well known data set which contains millions of images with many features.

It has been well know to be used on image classification techniques so that is why I chose it.

In [ ]:
# loading the renet model and downloading it
resnet_50 = tf.keras.applications.resnet50.ResNet50(include_top= False, weights= "imagenet")
resnet_50.trainable=False

include_top = False means that the last layer of the model will not be loaded. This means that as I want to use the model for my own classification task it will omit the final layer so that I can adjust it to my needs

weights = "imagenet" This uses the weights that are set from the imagenet data that the model has been trained on. I do not want to change these as the point of transfer learning is using a pre trained model with trained weights!

In [ ]:

inputs = keras.Input(shape=(64,64, 3)) # setting the input shape
augmented_inputs = data_augmentation(inputs)
# adding the resnet layers
layer = resnet_50(augmented_inputs)

layer = layers.BatchNormalization()(layer)

layer = layers.Activation('relu')(layer)


# then add a global max pooling layer
layer = layers.GlobalAveragePooling2D()(layer) # This adds max pooling to the model

# add dropout layer 
layer = layers.Dropout(0.3)(layer)

# adding last dense layer for categorization, using softmax for multi categorization
outputs = layers.Dense(5, activation="softmax")(layer) 
# model architecture

model = keras.Model(inputs=inputs, outputs=outputs, name="transfer_model")

# compiling model
model.compile(
        optimizer=keras.optimizers.Adam(learning_rate= 0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

model.summary()


I experimented with the model without any augmentation and drop out. There results were severe overfitting, so I decided to add them to the model to see if there was any improvements. 


By using the pre trained weights from the model means that the model has learnt from a mass of images. This has the capability of being able to take abstract features from images.   

I have also tried Batch normalization after the convolution layers of the Resnet model, this will hopefully improve the consistencies of the values within in the network. It seemed to have a very positive effect on the network. I didn't apply this on my original model, but I think if I did it would improve it much more. By adding the relu after it only passes the positive numbers to help the network learn better. 

I added a Global average pooling as this can help reduce the number of parameters instead of Max pooling and highlighting the most important features.





In [ ]:
check_filepath = "../transfer_model/checkpoint.model.keras"

model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=check_filepath, 
    monitor="val_accuracy", 
    mode="max", verbose=1, 
    save_best_only=True)

history = model.fit(train_images, train_labels, epochs=50, validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0) # test data results when run through the model
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

#visualizing training and validation results
plt.figure(figsize=(6, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss ")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()